# Comparing GMPEs Using Trellis Plots

### Manual Creation of Trellis Plots

In [ ]:
%load_ext autoreload
%autoreload 2
import warnings; warnings.filterwarnings("ignore")

In [ ]:
# Import the modules
%matplotlib inline
import numpy as np
import smtk.trellis.trellis_plots as trpl
from openquake.hazardlib.gsim import get_available_gsims

In [ ]:
get_available_gsims().keys()

In [ ]:
# Set up the configuration
GMPES = get_available_gsims()
gmpe_list = ["AkkarBommer2010", 
             "AkkarCagnan2010", 
             "AkkarEtAlRjb2014", 
             "BooreAtkinson2008", 
             "ChiouYoungs2008",
             "ZhaoEtAl2006Asc"]

imts = ["PGA", "SA(0.2)", "SA(1.0)", "SA(2.0)"]

params = {"ztor": 5.0,   # Top of rupture depth
         "hypo_depth": 10.0,   # Hypocentral depth
         "vs30": 800.0, # Vs30 for all sites
         "vs30measured": True, # Vs30 value is measured
         "z1pt0": 100.0, # Depth (m) to the 1.0 km/s Vs interface 
         "dip": 90.0,  # Vertical Fault
         "rake": 0.0 # Strike-slip fault
         }

In [ ]:
magnitudes = np.arange(4.5, 8.1, 0.1)
distances = {"repi": 20.0,
             "rhypo": 22.5,
             "rjb": 15.0,
             "rrup": 16.0,
             "rx": 15.0}

##### Example 1: Scaling of IMT with magnitude

In [ ]:
mag_imt_trellis = trpl.MagnitudeIMTTrellis(magnitudes, distances, gmpe_list, imts, params, figure_size=(7,5),
                                           filename="images/magnitude_imt_trellis_simple.pdf", filetype="pdf")


In [ ]:
mag_imt_trellis.pretty_print("data/magnitude_imt_trellis1.csv", sep=",")

In [ ]:
mag_imt_sigma_trellis = trpl.MagnitudeSigmaIMTTrellis(magnitudes, distances, gmpe_list, imts, params, 
                                                      stddevs="Total", figure_size=(7,5),
                                                      filename="images/magnitude_intra_sigma_imt_trellis_simple.pdf",
                                                      filetype="pdf")

In [ ]:
mag_imt_sigma_trellis.pretty_print("data/magnitude_imt_sigma_trellis1.csv", sep=",")

##### Example 2: Scaling of IMT with Distance

In [ ]:
magnitudes = 6.5
distances = {"repi": np.arange(0.0, 151.0, 1.0)}
distances["rhypo"] = np.sqrt(distances["repi"] ** 2.0 + params["hypo_depth"] ** 2)
distances["rjb"] = distances["repi"]
distances["rrup"] = np.sqrt(distances["rjb"] ** 2.0 + params["ztor"] ** 2)
distances["rx"] = distances["rjb"]


In [ ]:
dist_imt_trellis = trpl.DistanceIMTTrellis(magnitudes, distances, gmpe_list, imts, params,
                                           distance_type="rjb", plot_type="loglog",
                                           filename="images/magnitude_distance_imt_trellis_simple.pdf", filetype="pdf")

In [ ]:
dist_imt_trellis.pretty_print("data/distance_imt_trellis1.csv", sep=",")

In [ ]:
dist_imt_sigma_trellis = trpl.DistanceSigmaIMTTrellis(magnitudes, distances, gmpe_list, imts, params,
                                                      distance_type="rjb", plot_type="loglog",
                                                      filename="images/distance_total_sigma_imt_trellis_simple.pdf",
                                                      filetype="pdf")

In [ ]:
dist_imt_sigma_trellis.pretty_print("data/distance_imt_sigma_trellis1.csv", sep=",")

##### Example 3 : Response Spectra with Magnitude and Distance

In [ ]:
periods = [0.05, 0.075, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19,
           0.20, 0.22, 0.24, 0.26, 0.28, 0.30, 0.32, 0.34, 0.36, 0.38, 0.40, 0.42, 0.44, 0.46, 0.48, 0.5, 
           0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 
           1.9, 2.0, 3.0, 4.0, 5.0, 7.5, 10.0]

# Choose 4 magnitudes for comparison: 5, 6, 7, 8
magnitudes = np.array([5.0, 6.0, 7.0, 8.0])
# Choose 4 distances for comparison: 5., 20., 50., 100.
distances = {"repi": np.array([5.0, 20.0, 50.0, 100.0])}
distances["rhypo"] = np.sqrt(distances["repi"] ** 2.0 + params["hypo_depth"] ** 2)
distances["rjb"] = distances["repi"]
distances["rrup"] = np.sqrt(distances["rjb"] ** 2.0 + params["ztor"] ** 2)
distances["rx"] = distances["rjb"]

In [ ]:
mag_dist_spectra_trellis = trpl.MagnitudeDistanceSpectraTrellis(magnitudes, distances, gmpe_list,
                                                                periods, params, plot_type="loglog",
                                                                figure_size=(10,8),
                                                                filename="images/spectra_trellis_simple.pdf",
                                                                filetype="pdf")

In [ ]:
mag_dist_spectra_trellis.pretty_print("data/mag_dist_spectra_trellis.csv", sep=",")

In [ ]:
mag_dist_spectra_sigma = trpl.MagnitudeDistanceSpectraSigmaTrellis(magnitudes, distances, gmpe_list,
                                                                   periods, params, plot_type="loglog",
                                                                   figure_size=(10,8),
                                                                   filename="images/spectra_total_sigma_trellis_simple.pdf",
                                                                   filetype="pdf")

In [ ]:
mag_dist_spectra_sigma.pretty_print("data/mag_dist_spectra_sigma.csv", sep=",")

## Creation of a Trellis Plots from a Rupture-Site Configuration

##### Setting up the Rupture

In [ ]:
import smtk.trellis.configure as rcfg

In [ ]:
#A rupture is configured with the following properties

# The hypocentre location can be configured using the tuple (along-strike fraction, down-dip fraction)
# If not specified, the rupture will place the hypocentre in the centroid of the plane i.e.
hypo_loc = (0.5, 0.5) 


rupt1 = rcfg.GSIMRupture(magnitude=6.5,   # Moment magnitude
                         dip=45.0,   # Dip of Rupture
                         aspect=1.5, # Aspect Ratio of Rupture
                         rake=90.0,  # Rake of rupture 
                         ztor=0.0,  # Top of rupture depth
                         strike=0.0,  # Strike of rupture
                         hypocentre_location=hypo_loc   # Location of hypocentre within rupture plane
                         )

### Configuring the Sites

##### Set-up as a Mesh of Points

In [ ]:
_ = rupt1.get_target_sites_mesh(maximum_distance=200.0,
                                spacing = 5.0,
                                vs30 = 800.0)
rupt1.plot_model_configuration(marker_style=".", filename="images/rupt_config_regular_mesh.pdf", filetype="pdf")

In [ ]:
# If you are interested to see how two differen distance types compare in this scenario there is
# a built-in plotting function
rupt1.plot_distance_comparisons("rjb", "rrup", filename="images/rupt_distance_comp_1.pdf", filetype="pdf")

##### Set up as a Line of Points

In [ ]:
rupt2 = rcfg.GSIMRupture(magnitude=6.5,   # Moment magnitude
                         dip=45.0,   # Dip of Rupture
                         aspect=1.5, # Aspect Ratio of Rupture
                         rake=90.0,  # Rake of rupture 
                         ztor=0.0,  # Top of rupture depth
                         strike=0.0,  # Strike of rupture
                         hypocentre_location=hypo_loc   # Location of hypocentre within rupture plane
                         )
_ = rupt2.get_target_sites_line(200.0, 2.0, vs30=800.0, line_azimuth=90, as_log=False)
rupt2.plot_model_configuration(filename="images/rupt_config_line.pdf", filetype="pdf")

In [ ]:
rupt2.plot_distance_comparisons("repi", "rjb", filename="images/distance_config_2.pdf", filetype="pdf")

##### Set up as a single Point (useful for spectra comparisons!)

In [ ]:
rupt3 = rcfg.GSIMRupture(magnitude=6.5,   # Moment magnitude
                         dip=45.0,   # Dip of Rupture
                         aspect=1.5, # Aspect Ratio of Rupture
                         rake=90.0,  # Rake of rupture 
                         ztor=0.0,  # Top of rupture depth
                         strike=0.0,  # Strike of rupture
                         hypocentre_location=hypo_loc   # Location of hypocentre within rupture plane
                         )
_ = rupt3.get_target_sites_point(20.0,  # Distance (km) 
                                 "rjb",  # Distance Type
                                 vs30=800.0, 
                                 line_azimuth=90.)
rupt3.plot_model_configuration(filename="images/rupt_config_site.pdf", filetype="pdf")

### Create the Trellis Plots

In this example we will use just the line configuration


In [ ]:
rupt1 = rcfg.GSIMRupture(magnitude=6.5,   # Moment magnitude
                         dip=45.0,   # Dip of Rupture
                         aspect=1.5, # Aspect Ratio of Rupture
                         rake=90.0,  # Rake of rupture 
                         ztor=0.0,  # Top of rupture depth
                         strike=0.0,  # Strike of rupture
                         hypocentre_location=hypo_loc   # Location of hypocentre within rupture plane
                         )
# It is critical that before running the trellis plots, the site configuration must be run!!!!
_ = rupt1.get_target_sites_line(200.0, 1.0, vs30=800.0, line_azimuth=90)
rupt1.plot_model_configuration()

##### Example 1 - Distance Scaling with IMT

In [ ]:
trpl.DistanceIMTTrellis.from_rupture_model(rupt1, gmpe_list, imts, 
                                           filename="images/distance_trellis_rupt.pdf", filetype="pdf")

In [ ]:
trpl.DistanceSigmaIMTTrellis.from_rupture_model(rupt1, gmpe_list, imts, 
                                                filename="images/distance_sigma_trellis_rupt.pdf", filetype="pdf")

##### Distance Scaling along a different aziumth

In [ ]:
rupt2 = rcfg.GSIMRupture(magnitude=6.5,   # Moment magnitude
                         dip=45.0,   # Dip of Rupture
                         aspect=1.5, # Aspect Ratio of Rupture
                         rake=90.0,  # Rake of rupture 
                         ztor=0.0,  # Top of rupture depth
                         strike=0.0,  # Strike of rupture
                         hypocentre_location=hypo_loc   # Location of hypocentre within rupture plane
                         )
# It is critical that before running the trellis plots, the site configuration must be run!!!!
_ = rupt2.get_target_sites_line(200.0, 1.0, vs30=800.0, line_azimuth=230.0)
rupt2.plot_model_configuration(filename="images/rupt_config_footwall.pdf", filetype="pdf")

In [ ]:
trpl.DistanceIMTTrellis.from_rupture_model(rupt2, gmpe_list, imts,
                                           filename="images/distance_footwall_trellis_rupt.pdf", filetype="pdf")

In [ ]:
trpl.DistanceSigmaIMTTrellis.from_rupture_model(rupt2, gmpe_list, imts, 
                                                filename="images/distance_sigma_trellis_rupt.pdf", filetype="pdf")

##### Spectral Scaling IMT

In [ ]:
rupt3 = rcfg.GSIMRupture(magnitude=6.5,   # Moment magnitude
                         dip=45.0,   # Dip of Rupture
                         aspect=1.5, # Aspect Ratio of Rupture
                         rake=90.0,  # Rake of rupture 
                         ztor=0.0,  # Top of rupture depth
                         strike=0.0,  # Strike of rupture
                         hypocentre_location=hypo_loc   # Location of hypocentre within rupture plane
                         )
_ = rupt3.get_target_sites_point(20.0,  # Distance (km) 
                                 "rjb",  # Distance Type
                                 vs30=800.0, 
                                 line_azimuth=90.)
rupt3.plot_model_configuration()

In [ ]:
trpl.MagnitudeDistanceSpectraTrellis.from_rupture_model(rupt3, gmpe_list, periods,
                                                        filename="images/spectra_trellis_rupt.pdf", filetype="pdf")

In [ ]:
trpl.MagnitudeDistanceSpectraSigmaTrellis.from_rupture_model(rupt3, gmpe_list, periods,
                                                             filename="images/spectra_sigma_trellis_rupt.pdf", filetype="pdf")

##### With a different azimuth (now on the footwall!)

In [ ]:
rupt3 = rcfg.GSIMRupture(magnitude=6.5,   # Moment magnitude
                         dip=45.0,   # Dip of Rupture
                         aspect=1.5, # Aspect Ratio of Rupture
                         rake=90.0,  # Rake of rupture 
                         ztor=0.0,  # Top of rupture depth
                         strike=0.0,  # Strike of rupture
                         hypocentre_location=hypo_loc   # Location of hypocentre within rupture plane
                         )
_ = rupt3.get_target_sites_point(20.0,  # Distance (km) 
                                 "rjb",  # Distance Type
                                 vs30=800.0, 
                                 line_azimuth=220.)
rupt3.plot_model_configuration(filename="images/rupture_config_footwall_site.pdf", filetype="pdf")

In [ ]:
trpl.MagnitudeDistanceSpectraTrellis.from_rupture_model(rupt3, gmpe_list, periods,
                                                        filename="images/spectra_trellis_rupt_footwall.pdf",
                                                        filetype="pdf")

In [ ]:
trpl.MagnitudeDistanceSpectraSigmaTrellis.from_rupture_model(rupt3, gmpe_list, periods,
                                                             filename="images/spectra_sigma_trellis_rupt_footwall.pdf",
                                                             filetype="pdf")